In [ ]:
#PCA METHOD WITH REPORTING
import pandas as pd
import numpy as np
import math
import networkx as nx
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def reduce_pca_by_variance(data: np.ndarray, feature_names: list, variance_threshold: float):
    """
    Performs PCA on n-dimensional data, automatically selecting the minimum
    number of components to explain at least the `variance_threshold`.
    
    This modified version also prints the results of the reduction and
    the top 5 feature contributors for each component.

    Args:
        data: A (n_samples, n_features) NumPy array.
        feature_names: A list of strings corresponding to the feature columns
                       in `data`. (e.g., list(df.columns))
        variance_threshold: The target amount of variance to explain
                            (e.g., 0.95 for 95%).

    Returns:
        A tuple containing:
        - data_transformed (np.ndarray): The data projected onto the
                                         new component space.
        - fitted_pca (PCA): The fitted PCA object, which you can use
                            to inspect the number of components, etc.
        - explained_variance_list (list): A list of the variance explained
                                          by each component (e.g., [0.5, 0.3]).
    """
    
    if len(feature_names) != data.shape[1]:
        raise ValueError(f"Number of feature_names ({len(feature_names)}) does not "
                         f"match number of data columns ({data.shape[1]}).")

    # 1. Create a PCA object with the variance threshold.
    # By setting n_components to a float, PCA automatically finds
    # the components needed to explain that much variance.
    pca = PCA(n_components=variance_threshold)
    
    # 2. Create a pipeline to first scale the data, then run PCA.
    # Scaling is crucial for PCA to work correctly.
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', pca)
    ])
    
    # 3. Fit the pipeline to the data and transform it
    data_transformed = pipeline.fit_transform(data)
    
    # --- Report print statements ---
    
    # Get the original and new dimensions
    original_dimensions = data.shape[1]
    # We access the fitted pca object from step 2
    new_dimensions = pca.n_components_ 
    
    print("-" * 30)
    print("PCA Dimensionality Reduction Report")
    print("-" * 30)
    print(f"Original dimensions:   {original_dimensions}")
    print(f"New dimensions:        {new_dimensions}")
    print(f"Dimensions reduced by: {original_dimensions - new_dimensions}")
    print("\nVariance explained by each remaining component:")
    
    # pca.explained_variance_ratio_ is an array like [0.5, 0.3, 0.1]
    for i, variance in enumerate(pca.explained_variance_ratio_):
        print(f"  Principal Component {i+1}: {variance * 100:.2f}%")
        
    # Print total variance explained
    total_variance = np.sum(pca.explained_variance_ratio_)
    print(f"\nTotal variance explained: {total_variance * 100:.2f}%")
    print(f"(Target threshold was {variance_threshold * 100:.0f}%)")
    
    # --- New: Top 5 Contributors per Component Report ---
    print("\nTop 5 Contributors per Component:")
    
    # pca.components_ has shape (n_components, n_features)
    for i, component in enumerate(pca.components_):
        print(f"  --- Principal Component {i+1} ---")
        
        # Get indices of the top 5 absolute loadings
        # np.argsort returns indices of smallest to largest
        # We take the last 5, and then reverse them [::-1]
        top_5_indices = np.argsort(np.abs(component))[-5:][::-1]
        
        # Print the feature name and its loading (weight)
        for j, feature_index in enumerate(top_5_indices):
            feature_name = feature_names[feature_index]
            loading = component[feature_index]
            print(f"    {j+1}. {feature_name}: {loading:.4f}")
            
    print("-" * 30)
    
    # --- End of report ---
    
    # Get the list of explained variances
    explained_variance_list = pca.explained_variance_ratio_.tolist()
    
    # Return the new data, the fitted PCA object, and the list of variances
    return data_transformed, pca, explained_variance_list


# Processing

In [ ]:
# Get data to be a numpy array probably
df=pd.read_csv('consolidated_with_cd_118_116.csv')
data=np.array(df)
income_index = df.columns.get_loc('per_capita_income')
# Transform the income dimension by logarithmic scale
data[:, income_index] = np.log(data[:, income_index].astype(np.float64))
# Normalize data by dividing by standard deviation by dimension
for i in range(2,data.shape[1]-2):
    data[:, i] = data[:, i] / np.std(data[:, i])


## Weighting

In [ ]:
# Initilize weights, and scale data by weights
# Get column indices
poverty_index = df.columns.get_loc('avg_poverty_ratio')
employment_index = df.columns.get_loc('in_labor_force')
healthcare_index = df.columns.get_loc('Educational services, and health care and social assistance')
education_index = df.columns.get_loc('College or More')
commute_index = df.columns.get_loc('avg_commute_time')
housing_index = df.columns.get_loc('avg_housing_cost_burden')
race_indices = [df.columns.get_loc(col) for col in ['white', 'black', 'asian', 'native', 'pacific islander', 'other']]

# Apply dimension weights
data[:, income_index] *= 0.1225          # Economic Security - Income
data[:, poverty_index] *= 0.0525         # Economic Security - Poverty
data[:, employment_index] *= 0.07        # Economic Security - Employment
data[:, healthcare_index] *= 0.105       # Economic Security - Healthcare
data[:, education_index] *= 0.15         # Education
data[:, commute_index] *= 0.15           # Location Affordability - Transportation
data[:, housing_index] *= 0.15           # Location Affordability - Housing

# Race: distribute 0.2 across 6 race columns
for idx in race_indices:
    data[:, idx] *= 0.2 / 6  # 0.0333 per column

weights = np.array([
    0.1225,     # 1:  per_capita_income (Economic Security - Income)
    0.2/6,      # 2:  white (Cultural - Race)
    0.2/6,      # 3:  black (Cultural - Race)
    0.2/6,      # 4:  asian (Cultural - Race)
    0.2/6,      # 5:  native (Cultural - Race)
    0.2/6,      # 6:  pacific islander (Cultural - Race)
    0.2/6,      # 7:  other (Cultural - Race)
    0.0,        # 8:  Under High School (not weighted)
    0.0,        # 9:  High School (No College Degree) (not weighted)
    0.15,       # 10: College or More (Education)
    0.0,        # 11: Agriculture (not weighted)
    0.0,        # 12: Construction_and_manufacturing (not weighted)
    0.0,        # 13: trade (not weighted)
    0.0,        # 14: Transportation and warehousing (not weighted)
    0.0,        # 15: nerds (not weighted)
    0.105,      # 16: Educational services, and health care (Economic Security - Healthcare)
    0.0,        # 17: finance_inurance_and_realty (not weighted)
    0.0,        # 18: other_services (not weighted)
    0.07,       # 19: in_labor_force (Economic Security - Employment)
    0.0,        # 20: out_labor_force (not weighted)
    0.15,       # 21: avg_commute_time (Location Affordability - Transportation)
    0.15,       # 22: avg_housing_cost_burden (Location Affordability - Housing)
    0.0525,     # 23: avg_poverty_ratio (Economic Security - Poverty)
])


In [ ]:
# Perform PCA, and project onto the top N dimensions so that they explain 50% of the variance
new_data,pca,var_explained = reduce_pca_by_variance(data[:,2:-2], list(df.columns)[2:-2], variance_threshold=0.5)

------------------------------
PCA Dimensionality Reduction Report
------------------------------
Original dimensions:   23
New dimensions:        6
Dimensions reduced by: 17

Variance explained by each remaining component:
  Principal Component 1: 16.07%
  Principal Component 2: 10.11%
  Principal Component 3: 8.43%
  Principal Component 4: 6.79%
  Principal Component 5: 6.32%
  Principal Component 6: 5.67%

Total variance explained: 53.38%
(Target threshold was 50%)

Top 5 Contributors per Component:
  --- Principal Component 1 ---
    1. College or More: 0.4448
    2. avg_poverty_ratio: 0.3872
    3. Under High School: -0.3443
    4. per_capita_income: 0.2592
    5. out_labor_force: -0.2370
  --- Principal Component 2 ---
    1. out_labor_force: 0.4101
    2. in_labor_force: -0.4101
    3. white: 0.3803
    4. pacific islander: 0.3752
    5. Agriculture, forestry, fishing and hunting, and mining: 0.2772
  --- Principal Component 3 ---
    1. High School (No College Degree): 0.4196
 

As shown, the first component has a distinct identity, namely education, income and poverty, part of the most important factors typically considered as determinant of a person's sociol-economic status. The rest of the components are a mixed of race, education, occupation, and comute time.

# Network initialization

In [ ]:
def create_dimension_layered_knn(data, dimension_weights, k=10):
    """
    Create multi-layer network where each dimension has its own KNN graph.
    """
    G = nx.Graph()
    
    # Add nodes
    for i in range(data.shape[0]):
        G.add_node(i)
    
    # For each dimension, create KNN graph
    for dim in range(data.shape[1]):
        dim_weight = dimension_weights[dim]
        
        if dim_weight == 0:
            continue
        
        # Get this dimension's values (1D)
        dim_data = data[:, dim].reshape(-1, 1)
        
        # Build KNN graph for THIS dimension only
        from sklearn.neighbors import NearestNeighbors
        nbrs = NearestNeighbors(n_neighbors=k)
        nbrs.fit(dim_data)
        distances, indices = nbrs.kneighbors(dim_data)
        
        # Add edges weighted by dimension importance
        for i in range(len(data)):
            for j, neighbor in enumerate(indices[i]):
                if i != neighbor:
                    if G.has_edge(i, neighbor):
                        G[i][neighbor]['weight'] += dim_weight  # Accumulate
                    else:
                        G.add_edge(i, neighbor, weight=dim_weight)
    
    return G


In [ ]:

G = create_dimension_layered_knn(new_data, dimension_weights=var_explained, k=10)
G_uni= nx.Graph() #Create a simple unweighted copy for baseline
G_uni.add_nodes_from(G.nodes(data=True))
G_uni.add_edges_from(G.edges())


## Geographic weighting

In [ ]:
import pickle
from scipy.sparse import load_npz

# Create graph
Geo = G.copy()
#Gring_geo=Gring.copy()
# Load adjacency matrix
adj_matrix = load_npz('adjacency_queen_matrix.npz')

# Load mappings
with open('adjacency_queen_mappings.pkl', 'rb') as f:
    mappings = pickle.load(f)
index_to_geoid = mappings['index_to_geoid']

# Get edges from sparse matrix
rows, cols = adj_matrix.nonzero()

# Add edges with your weight
weight = 2  # Change this to your desired weight

for i, j in zip(rows, cols):
    if i < j:  # Only add each edge once (undirected)
        geoid1 = index_to_geoid[i]
        geoid2 = index_to_geoid[j]
        Geo.add_edge(i, j, weight=weight)
        #Gring_geo.add_edge(i, j, weight=weight)


# Grouping

In [ ]:
# K means clustering function
from sklearn.cluster import KMeans

def get_kmeans_partition(data: np.ndarray, weights, n_clusters=14):
    """
    Runs K-means clustering on the input data and returns the loss
    (inertia) and a partition of the data indices by cluster.

    Args:
        data: A (n_samples, n_features) NumPy array.
        n_clusters: The number of clusters (k).

    Returns:
        A tuple containing:
        - loss (float): The inertia (Within-Cluster Sum of Squares).
        - partitions (dict): A dictionary where keys are cluster IDs (0 to k-1)
                             and values are lists of original data indices
                             belonging to that cluster.
    """
    
    # 1. Initialize and fit the K-means model
    # n_init=10 runs the algorithm 10 times and picks the best result
    # random_state=42 ensures the result is reproducible
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    for i in range(len(weights)):
        data[:, i] = data[:, i] * weights[i]
    kmeans.fit(data)

    # 2. Get the loss (inertia)
    # .inertia_ is the WCSS (Within-Cluster Sum of Squares)
    loss = kmeans.inertia_

    # 3. Get the cluster assignment for each data point
    # .labels_ is an array like [0, 1, 1, 0, 2, ...]
    labels = kmeans.labels_

    # 4. Create the partition of indices
    partitions = {i: [] for i in range(n_clusters)}
    for index, cluster_id in enumerate(labels):
        partitions[cluster_id].append(index)

    return loss, partitions

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

def calculate_kmeans_loss_for_partition(data: np.ndarray, partition: list) -> float:
    """
    Calculates the K-means "loss" (Inertia, or Within-Cluster Sum of Squares)
    for a given dataset and a user-provided partition.

    Args:
        data: A (n_samples, n_features) NumPy array.
        partition: A list of lists, where each inner list contains the
                   *indices* (row numbers) of the data points belonging
                   to that cluster.
                   Example: [[0, 1, 4], [2, 3, 5]]

    Returns:
        total_loss (float): The total K-means loss (Inertia) for this partition.
    """
    
    total_loss = 0.0

    # Iterate over each cluster (which is a list of indices)
    for indices in partition:
        
        # 1. Get all data points belonging to this cluster
        # Using [indices, :] selects all rows whose index is in the list
        cluster_points = data[indices, :]
        
        # Handle empty clusters (their loss is 0)
        if cluster_points.shape[0] == 0:
            continue
            
        # 2. Calculate the "true" centroid (mean) for this cluster
        # axis=0 calculates the mean of each *column* (feature)
        centroid = np.mean(cluster_points, axis=0)
        
        # 3. Calculate the sum of squared distances from each point to the centroid
        #    - (cluster_points - centroid) uses broadcasting to get distance vectors
        #    - (** 2) squares all distances
        #    - np.sum(...) sums all squared distances into a single number
        cluster_loss = np.sum((cluster_points - centroid) ** 2)
        
        # 4. Add this cluster's loss to the total
        total_loss += cluster_loss
        
    return total_loss




k = 14

# 3. Run the function
total_loss, index_partitions = get_kmeans_partition(new_data,weights=var_explained, n_clusters=k)

index_partitions=[index_partitions[i] for i in range(k)]

In [ ]:
partG=nx.algorithms.community.louvain_communities(G,weight='weight',resolution=0.75)
print("Number of Louvain communities G:",len(partG))
partGeo=nx.algorithms.community.louvain_communities(Geo,weight='weight',resolution=0.8)
print("Number of Louvain communities Geo:",len(partGeo))
partG_uni=nx.algorithms.community.louvain_communities(G_uni,weight='weight',resolution=0.95)
print("Number of Louvain communities G_uni:",len(partG_uni))



Number of Louvain communities G: 15
Number of Louvain communities Geo: 21
Number of Louvain communities G_uni: 11


In [ ]:
# Parse recent districting
part116=[[] for i in range(13)]
part118=[[] for i in range(14)]
for i in range(data.shape[0]):
    cd116=int(data[i,-1])-1
    cd118=int(data[i,-2])-1
    part116[cd116].append(i)
    part118[cd118].append(i)

In [ ]:
print("\n 10Graph Comparisons")
print("Louvain modularity G:",nx.algorithms.community.modularity(G, partG, weight='weight'))
print("Louvain modularity Geo partition on G:",nx.algorithms.community.modularity(G, partGeo, weight='weight'))
print("Louvain modularity Uni partition on G:",nx.algorithms.community.modularity(G, partG_uni, weight='weight'))

print("\n K Means Comparisons")
print("K means modularity on G:",nx.algorithms.community.modularity(G, index_partitions, weight='weight'))

print("\nReal district modularity on G")
print("116 partition on G:",nx.algorithms.community.modularity(G, part116, weight='weight'))
print("118 partition on G:",nx.algorithms.community.modularity(G, part118, weight='weight'))


 10Graph Comparisons
Louvain modularity G: 0.28617087673006536
Louvain modularity Geo partition on G: 0.008999721432192267
Louvain modularity Uni partition on G: 0.1499531018144754

 K Means Comparisons
K means modularity on G: 0.10560605648755732

Real district modularity on G
116 partition on G: 0.00854993617357976
118 partition on G: 0.008228473361896613


In [ ]:

# 1. Load your real data (as you were before)

# 2. Get your partition
# (e.g., from K-means, Louvain, or any other method)
# This is just a placeholder example!
# You would have your *real* partition here.

# 3. Set the path to the shapefile you downloaded
# (Update this path to where you unzipped the file)
SHAPEFILE = "tl_2020_37_tract/tl_2020_37_tract.shp"

# 4. Call the plotting function
# from plot_geo import *
# plot_partition_geographically(data, partGeo, SHAPEFILE)

In [ ]:
# need initial partition to have 14 groups

In [ ]:
from typing import List, Set, Dict, Tuple
import time

# --- Helper Functions ---

def partition_to_map(partition: List[Set[int]]) -> Dict[int, int]:
    """Converts a list of sets to a dictionary mapping node -> community_index."""
    node_to_community = {}
    for i, community_set in enumerate(partition):
        for node in community_set:
            node_to_community[node] = i
    return node_to_community

def check_contiguity(Geo: nx.Graph, old_community_idx: int, prospective_partition: List[Set[int]]) -> bool:
    """
    Checks if the community at old_community_idx remains contiguous after a move.
    
    Args:
        Geo: The geographical adjacency graph.
        old_community_idx: The index of the community to check.
        prospective_partition: The partition *after* the move has been simulated.
        
    Returns:
        True if the community remains contiguous, False otherwise.
    """
    
    remaining_nodes = prospective_partition[old_community_idx]
    
    if len(remaining_nodes) <= 1:
        return True
        
    # Get the first node to start the check
    start_node = next(iter(remaining_nodes))
    
    # Perform a graph traversal (BFS) on the subgraph induced by remaining_nodes
    # use the Geo graph's edges for contiguity.
    
    # Create a set of reachable nodes
    reachable = set()
    queue = [start_node]
    
    while queue:
        current = queue.pop(0)
        if current not in reachable:
            reachable.add(current)
            # Check neighbors in the Geo graph
            for neighbor in Geo.neighbors(current):
                # Only follow edges where the neighbor is still in the community
                if neighbor in remaining_nodes and neighbor not in reachable:
                    queue.append(neighbor)
                    
    # Check if all remaining nodes were reached
    return len(reachable) == len(remaining_nodes)

# --- Core Optimization Function with Sequential Swap and Delta Q ---
def optimized_sequential_swap(G: nx.Graph, Geo: nx.Graph, initial_partition: List[Set[int]], N_iterations: int) -> Tuple[List[Set[int]], float]:
    """
    Performs local search optimization using a sequential, targeted two-node swap.
    In each iteration, it finds the single best swap across all districts and executes it.
    """
    
    # 1. Pre-calculate initial community properties
    best_part = [set(comm) for comm in initial_partition] # current best partition
    node_to_community_map = partition_to_map(best_part) # convert to map for better searching
    
    # Total edge weight of the graph G (2m)
    total_weight_2m = G.size(weight='weight') * 2
    
    # Calculate initial community properties for Delta Q calculation (change in modularity)
    community_properties = {}
    for i, community in enumerate(best_part):
        subgraph = G.subgraph(community)
        total_internal_weight = subgraph.size(weight='weight')
        total_degree_sum = sum(G.degree(node, weight='weight') for node in community)
        
        community_properties[i] = {
            'internal_weight': total_internal_weight,
            'degree_sum': total_degree_sum
        }
    
    # Calculate initial modularity score
    modularity_part = tuple(frozenset(s) for s in best_part if s)
    best_score = nx.algorithms.community.modularity(G, modularity_part, weight='weight')

    total_duration = 0.0
    
    print(f"Starting optimization with Modularity Score: {best_score:.4f}")
    
    # --- Main Optimization Loop ---
    for i in range(N_iterations):
        start_time = time.time()
        print(f"--- Iteration {i+1}/{N_iterations} ---")
        
        best_gain = 0.0
        best_swap = None  # Stores: (node_A, node_B, comm_A_idx, comm_B_idx)
        
        # 2. Iterate through each district (community)
        # We use a list of indices to ensure we iterate over all districts
        district_indices = list(range(len(best_part)))
        
        # 3. Find the single best swap across ALL districts
        for comm_A_idx in district_indices:
            
            # Get all nodes in the current district
            district_A_nodes = best_part[comm_A_idx]
            
            # Identify all potential swap candidates involving a node from comm_A_idx
            # A swap candidate is a pair (A, B) where A is in comm_A_idx and B is adjacent to A
            # and B is in a different community comm_B_idx.
            
            # Iterate over all nodes A in the current district
            for node_A in district_A_nodes:
                
                # Iterate over all geographical neighbors of A
                for node_B in Geo.neighbors(node_A):
                    
                    comm_B_idx = node_to_community_map.get(node_B)
                    
                    # Check if B is in a different community
                    if comm_B_idx is not None and comm_B_idx != comm_A_idx:
                        
                        # Delta Q change in modularity calculation
                        
                        # 1. Calculate gain for moving A from comm_A to comm_B
                        k_A = G.degree(node_A, weight='weight')
                        k_A_in_B = sum(G[node_A][neighbor]['weight'] 
                                       for neighbor in G.neighbors(node_A) 
                                       if node_to_community_map.get(neighbor) == comm_B_idx)
                        
                        sigma_tot_A = community_properties[comm_A_idx]['degree_sum']
                        sigma_tot_B = community_properties[comm_B_idx]['degree_sum']
                        
                        # Delta Q for A moving to B (ignoring B)
                        gain_A = (k_A_in_B / total_weight_2m) - (k_A * sigma_tot_B / (total_weight_2m ** 2))
                        
                        # 2. Calculate gain for moving B from comm_B to comm_A
                        k_B = G.degree(node_B, weight='weight')
                        k_B_in_A = sum(G[node_B][neighbor]['weight'] 
                                       for neighbor in G.neighbors(node_B) 
                                       if node_to_community_map.get(neighbor) == comm_A_idx)
                        
                        # Delta Q for B moving to A (ignoring A)
                        gain_B = (k_B_in_A / total_weight_2m) - (k_B * sigma_tot_A / (total_weight_2m ** 2))
                        
                        # 3. Total Swap Gain (Sum of individual gains + correction)
                        w_AB = G.get_edge_data(node_A, node_B, default={'weight': 0.0})['weight']
                        total_gain = gain_A + gain_B + (w_AB / total_weight_2m)
                        
                        # --- B. Contiguity Check  ---
                        if total_gain > best_gain:
                            # Simulate the swap for the contiguity check
                            prospective_part = [set(comm) for comm in best_part]
                            
                            # A leaves comm_A, B leaves comm_B
                            prospective_part[comm_A_idx].remove(node_A)
                            prospective_part[comm_B_idx].remove(node_B)
                            
                            # A enters comm_B, B enters comm_A
                            prospective_part[comm_B_idx].add(node_A)
                            prospective_part[comm_A_idx].add(node_B)
                            
                            # Check if BOTH communities remain contiguous
                            contiguity_A = check_contiguity(Geo, comm_A_idx, prospective_part)
                            contiguity_B = check_contiguity(Geo, comm_B_idx, prospective_part)
                            
                            if contiguity_A and contiguity_B:
                                # 4. If the gain is the best so far, record the swap
                                best_gain = total_gain
                                best_swap = (node_A, node_B, comm_A_idx, comm_B_idx)

        # 5. Apply the single best swap for this iteration
        if best_gain > 0:
            node_A, node_B, comm_A_idx, comm_B_idx = best_swap
            
            # Update partition sets
            best_part[comm_A_idx].remove(node_A)
            best_part[comm_B_idx].remove(node_B)
            best_part[comm_B_idx].add(node_A)
            best_part[comm_A_idx].add(node_B)
            
            # Update node-to-community map
            node_to_community_map[node_A] = comm_B_idx
            node_to_community_map[node_B] = comm_A_idx
            
            # Re-calculate properties for the two affected communities
            for idx in [comm_A_idx, comm_B_idx]:
                community = best_part[idx]
                subgraph = G.subgraph(community)
                community_properties[idx]['internal_weight'] = subgraph.size(weight='weight')
                community_properties[idx]['degree_sum'] = sum(G.degree(node, weight='weight') for node in community)
            
            # Update total modularity score
            best_score += best_gain
            
            print(f"  Swap accepted! Node {node_A} moved from {comm_A_idx} to {comm_B_idx}, and Node {node_B} moved from {comm_B_idx} to {comm_A_idx}.")
            print(f"  Modularity increased by: {best_gain:.6f}. New score: {best_score:.4f}")
            
        else:
            print("\nNo local swap increased modularity while maintaining contiguity. Optimization finished.")
            break
            
        end_time = time.time()
        duration = end_time - start_time
        print(f"  Iteration {i+1} took: {duration:.2f} seconds")
        total_duration += duration
            
    print("\n--- Final Results ---")
    print(f"Optimal Modularity Score (after local swaps): {best_score:.4f}")
    return best_part, best_score, total_duration

# --- Example Usage (Requires G, Geo, and partGeo to be defined in the notebook) ---
# best_partition, final_score = optimized_local_search(G, Geo, partGeo, N_iterations=100)
# print(f"Final Partition Size: {len(best_partition)}")
# print(f"Final Modularity Score: {final_score}")

In [ ]:
best_partition, final_score, duration = optimized_local_search(G, Geo, partGeo, N_iterations=10)
print(f"Final Partition Size: {len(best_partition)}")
print(f"Final Modularity Score: {final_score}")
print(f"Optimization Time: {duration}")

In [ ]:
def is_contiguous(G: nx.Graph, partition: List) -> bool:
    for part in partition:
        if not nx.is_connected(G.subgraph(part)):
            return False
    return True